In [2]:
print("hi");

hi


In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import tkinter.scrolledtext as scrolledtext
import time
import threading
import os
import shutil
import ast

# --------------------- Safe evaluator for calculator ---------------------
class SafeEvaluator(ast.NodeVisitor):
    ALLOWED_NODES = (
        ast.Expression, ast.BinOp, ast.UnaryOp, ast.Constant,
        ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.Mod, ast.USub, ast.UAdd,
        ast.FloorDiv, ast.LShift, ast.RShift, ast.BitOr, ast.BitXor, ast.BitAnd
    )

    def visit(self, node):
        if not isinstance(node, self.ALLOWED_NODES):
            raise ValueError(f"Unsupported expression: {type(node).__name__}")
        return super().visit(node)

    def eval(self, expr: str):
        parsed = ast.parse(expr, mode='eval')
        self.visit(parsed)
        return eval(compile(parsed, '<string>', 'eval'))

_evaluator = SafeEvaluator()

# --------------------- Main Application ---------------------
class ProductivityApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Personal Productivity Suite")
        self.geometry("900x600")
        self.minsize(800, 500)

        self.style = ttk.Style(self)
        # Use default theme; user can change

        self.notebook = ttk.Notebook(self)
        self.notebook.pack(fill='both', expand=True)

        self._create_calculator_tab()
        self._create_notes_tab()
        self._create_timer_tab()
        self._create_file_organizer_tab()

    # ---------------- Calculator ----------------
    def _create_calculator_tab(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="Calculator")

        entry_frame = ttk.Frame(frame)
        entry_frame.pack(fill='x', padx=10, pady=10)

        self.calc_var = tk.StringVar()
        self.calc_entry = ttk.Entry(entry_frame, textvariable=self.calc_var, font=(None, 16))
        self.calc_entry.pack(side='left', fill='x', expand=True)
        self.calc_entry.bind('<Return>', lambda e: self.calculate())

        btn_eval = ttk.Button(entry_frame, text="=", command=self.calculate)
        btn_eval.pack(side='left', padx=5)

        btn_clear = ttk.Button(entry_frame, text="Clear", command=lambda: self.calc_var.set(''))
        btn_clear.pack(side='left')

        # simple keypad
        keypad = ttk.Frame(frame)
        keypad.pack(padx=10, pady=10)

        keys = [
            '7','8','9','/','%'
            ,'4','5','6','*','**'
            ,'1','2','3','-','('
            ,'0','.','=','+',')'
        ]
        r = 0
        c = 0
        for k in keys:
            def cmd(x=k):
                if x == '=':
                    self.calculate()
                else:
                    self.calc_var.set(self.calc_var.get() + x)
            b = ttk.Button(keypad, text=k, command=cmd, width=6)
            b.grid(row=r, column=c, padx=3, pady=3)
            c += 1
            if c >= 5:
                c = 0
                r += 1

    def calculate(self):
        expr = self.calc_var.get().strip()
        if not expr:
            return
        try:
            result = _evaluator.eval(expr)
            self.calc_var.set(str(result))
        except Exception as e:
            messagebox.showerror("Calculator Error", f"Invalid expression:\n{e}")

    # ---------------- Notes ----------------
    def _create_notes_tab(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="Notes")

        toolbar = ttk.Frame(frame)
        toolbar.pack(fill='x', padx=8, pady=6)

        btn_new = ttk.Button(toolbar, text="New", command=self.notes_new)
        btn_open = ttk.Button(toolbar, text="Open...", command=self.notes_open)
        btn_save = ttk.Button(toolbar, text="Save", command=self.notes_save)
        btn_saveas = ttk.Button(toolbar, text="Save As...", command=self.notes_save_as)

        for w in (btn_new, btn_open, btn_save, btn_saveas):
            w.pack(side='left', padx=3)

        autosave_lbl = ttk.Label(toolbar, text="Autosave:")
        autosave_lbl.pack(side='left', padx=(12,0))
        self.autosave_var = tk.BooleanVar(value=False)
        autosave_chk = ttk.Checkbutton(toolbar, variable=self.autosave_var, command=self._toggle_autosave)
        autosave_chk.pack(side='left')

        self.notes_text = scrolledtext.ScrolledText(frame, wrap='word', font=(None, 12))
        self.notes_text.pack(fill='both', expand=True, padx=8, pady=6)
        self.current_notes_path = None

        # autosave every 30 seconds if enabled
        self._autosave_thread = None
        self._stop_autosave = threading.Event()

    def notes_new(self):
        if self._notes_dirty():
            if not messagebox.askyesno('Unsaved', 'Discard current notes and create new?'):
                return
        self.notes_text.delete('1.0', tk.END)
        self.current_notes_path = None

    def notes_open(self):
        path = filedialog.askopenfilename(filetypes=[('Text files','*.txt'), ('All files','*.*')])
        if not path:
            return
        try:
            with open(path, 'r', encoding='utf-8') as f:
                text = f.read()
            self.notes_text.delete('1.0', tk.END)
            self.notes_text.insert(tk.END, text)
            self.current_notes_path = path
        except Exception as e:
            messagebox.showerror('Open Error', f'Could not open file:\n{e}')

    def notes_save(self):
        if not self.current_notes_path:
            return self.notes_save_as()
        try:
            with open(self.current_notes_path, 'w', encoding='utf-8') as f:
                f.write(self.notes_text.get('1.0', tk.END))
            messagebox.showinfo('Saved', f'Saved to {self.current_notes_path}')
        except Exception as e:
            messagebox.showerror('Save Error', f'Could not save:\n{e}')

    def notes_save_as(self):
        path = filedialog.asksaveasfilename(defaultextension='.txt', filetypes=[('Text files','*.txt'), ('All files','*.*')])
        if not path:
            return
        try:
            with open(path, 'w', encoding='utf-8') as f:
                f.write(self.notes_text.get('1.0', tk.END))
            self.current_notes_path = path
            messagebox.showinfo('Saved', f'Saved to {path}')
        except Exception as e:
            messagebox.showerror('Save Error', f'Could not save:\n{e}')

    def _notes_dirty(self):
        return bool(self.notes_text.get('1.0', tk.END).strip())

    def _toggle_autosave(self):
        if self.autosave_var.get():
            self._stop_autosave.clear()
            self._autosave_thread = threading.Thread(target=self._autosave_loop, daemon=True)
            self._autosave_thread.start()
        else:
            self._stop_autosave.set()

    def _autosave_loop(self):
        while not self._stop_autosave.is_set():
            time.sleep(30)
            try:
                tmp = os.path.join(os.path.expanduser('~'), '.productivity_autosave.txt')
                with open(tmp, 'w', encoding='utf-8') as f:
                    f.write(self.notes_text.get('1.0', tk.END))
            except Exception:
                pass

    # ---------------- Timer ----------------
    def _create_timer_tab(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="Timer")

        left = ttk.Frame(frame)
        left.pack(side='left', fill='both', expand=True, padx=8, pady=8)

        # Stopwatch
        sw_frame = ttk.LabelFrame(left, text='Stopwatch')
        sw_frame.pack(fill='x', pady=6)

        self.sw_time_var = tk.StringVar(value='00:00:00.00')
        ttk.Label(sw_frame, textvariable=self.sw_time_var, font=(None, 18)).pack(pady=6)
        sw_buttons = ttk.Frame(sw_frame)
        sw_buttons.pack()
        self._sw_running = False
        self._sw_start_ts = None
        self._sw_elapsed = 0.0

        ttk.Button(sw_buttons, text='Start', command=self._sw_start).pack(side='left', padx=4)
        ttk.Button(sw_buttons, text='Stop', command=self._sw_stop).pack(side='left', padx=4)
        ttk.Button(sw_buttons, text='Reset', command=self._sw_reset).pack(side='left', padx=4)

        # Countdown
        cd_frame = ttk.LabelFrame(left, text='Countdown')
        cd_frame.pack(fill='x', pady=6)

        ttk.Label(cd_frame, text='Enter seconds:').pack(side='left', padx=8)
        self.cd_entry = ttk.Entry(cd_frame, width=10)
        self.cd_entry.pack(side='left')
        ttk.Button(cd_frame, text='Start', command=self._cd_start).pack(side='left', padx=4)
        ttk.Button(cd_frame, text='Stop', command=self._cd_stop).pack(side='left', padx=4)
        self.cd_time_var = tk.StringVar(value='00:00:00')
        ttk.Label(cd_frame, textvariable=self.cd_time_var, font=(None, 14)).pack(pady=6)

        self._cd_remaining = 0
        self._cd_running = False

    # Stopwatch methods
    def _sw_start(self):
        if not self._sw_running:
            self._sw_running = True
            self._sw_start_ts = time.time() - self._sw_elapsed
            self._update_stopwatch()

    def _sw_stop(self):
        if self._sw_running:
            self._sw_running = False
            self._sw_elapsed = time.time() - self._sw_start_ts

    def _sw_reset(self):
        self._sw_running = False
        self._sw_start_ts = None
        self._sw_elapsed = 0.0
        self.sw_time_var.set('00:00:00.00')

    def _update_stopwatch(self):
        if not self._sw_running:
            return
        elapsed = time.time() - self._sw_start_ts
        h = int(elapsed // 3600)
        m = int((elapsed % 3600) // 60)
        s = elapsed % 60
        self.sw_time_var.set(f"{h:02d}:{m:02d}:{s:05.2f}")
        self.after(50, self._update_stopwatch)

    # Countdown methods
    def _cd_start(self):
        if self._cd_running:
            return
        try:
            secs = float(self.cd_entry.get())
            if secs <= 0:
                raise ValueError('Must be positive')
        except Exception:
            messagebox.showerror('Input Error', 'Enter a positive number of seconds')
            return
        self._cd_remaining = int(secs)
        self._cd_running = True
        self._run_countdown()

    def _cd_stop(self):
        self._cd_running = False

    def _run_countdown(self):
        if not self._cd_running:
            return
        if self._cd_remaining <= 0:
            self.cd_time_var.set('00:00:00')
            self._cd_running = False
            messagebox.showinfo('Countdown', 'Time is up!')
            return
        hrs = self._cd_remaining // 3600
        mins = (self._cd_remaining % 3600) // 60
        secs = self._cd_remaining % 60
        self.cd_time_var.set(f"{hrs:02d}:{mins:02d}:{secs:02d}")
        self._cd_remaining -= 1
        self.after(1000, self._run_countdown)

    # ---------------- File Organizer ----------------
    def _create_file_organizer_tab(self):
        frame = ttk.Frame(self.notebook)
        self.notebook.add(frame, text="File Organizer")

        top = ttk.Frame(frame)
        top.pack(fill='x', padx=8, pady=8)

        ttk.Label(top, text='Source folder:').pack(side='left')
        self.src_var = tk.StringVar()
        src_entry = ttk.Entry(top, textvariable=self.src_var)
        src_entry.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(top, text='Browse', command=self._choose_source).pack(side='left', padx=4)
        ttk.Button(top, text='Preview', command=self._preview_organize).pack(side='left', padx=4)
        ttk.Button(top, text='Organize', command=self._execute_organize).pack(side='left', padx=4)

        mid = ttk.Frame(frame)
        mid.pack(fill='both', expand=True, padx=8, pady=6)

        self.organizer_text = scrolledtext.ScrolledText(mid, height=10)
        self.organizer_text.pack(fill='both', expand=True)

        bottom = ttk.Frame(frame)
        bottom.pack(fill='x', padx=8, pady=6)
        ttk.Label(bottom, text='Pattern: Move files into folders by extension (e.g. .pdf -> pdf/)').pack(side='left')

    def _choose_source(self):
        path = filedialog.askdirectory()
        if path:
            self.src_var.set(path)

    def _preview_organize(self):
        path = self.src_var.get().strip()
        if not path or not os.path.isdir(path):
            messagebox.showerror('Folder Error', 'Please choose a valid source folder')
            return
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        preview = []
        for f in files:
            ext = os.path.splitext(f)[1].lower().lstrip('.') or 'noext'
            dest = os.path.join(path, ext)
            preview.append(f"{f} -> {ext}/")
        self.organizer_text.delete('1.0', tk.END)
        if not preview:
            self.organizer_text.insert(tk.END, 'No files found in folder.')
        else:
            self.organizer_text.insert(tk.END, '\n'.join(preview))

    def _execute_organize(self):
        path = self.src_var.get().strip()
        if not path or not os.path.isdir(path):
            messagebox.showerror('Folder Error', 'Please choose a valid source folder')
            return
        if not messagebox.askyesno('Confirm', f'Organize files in {path}? This will move files into subfolders.'):
            return
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        moved = []
        errors = []
        for f in files:
            src = os.path.join(path, f)
            ext = os.path.splitext(f)[1].lower().lstrip('.') or 'noext'
            dest_dir = os.path.join(path, ext)
            try:
                os.makedirs(dest_dir, exist_ok=True)
                shutil.move(src, os.path.join(dest_dir, f))
                moved.append(f)
            except Exception as e:
                errors.append((f, str(e)))
        out = []
        if moved:
            out.append('Moved files:')
            out.extend(moved)
        if errors:
            out.append('\nErrors:')
            out.extend([f'{a}: {b}' for a,b in errors])
        if not out:
            out = ['No files moved.']
        self.organizer_text.delete('1.0', tk.END)
        self.organizer_text.insert(tk.END, '\n'.join(out))


if __name__ == '__main__':
    app = ProductivityApp()
    app.mainloop()
